<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [2]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'./dataset/genres/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [4]:
file = open('dataset/data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset/data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [5]:
data = pd.read_csv('dataset/data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00043.au,0.399025,0.127311,2155.654923,2372.403604,5012.019693,0.087165,-109.165355,100.621500,-8.614721,...,6.585774,-8.642621,4.912259,-15.442804,1.538750,-6.732474,1.417774,-3.961750,3.287460,blues
1,blues.00012.au,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,-15.438986,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
2,blues.00026.au,0.278484,0.076970,1198.607665,1573.308974,2478.376680,0.051988,-284.819504,108.785628,9.131956,...,-3.303735,1.601561,2.660517,3.323455,3.258920,-4.551106,0.493845,5.937066,3.231544,blues
3,blues.00077.au,0.408876,0.243217,2206.771246,2191.473506,4657.388504,0.111526,-29.010990,104.532914,-30.974207,...,10.786454,-10.558812,6.877709,-10.294858,6.967845,-10.256100,0.705014,-6.000722,1.348955,blues
4,blues.00084.au,0.396258,0.235238,2061.150735,2085.159448,4221.149475,0.113397,-38.965941,112.039843,-31.817035,...,13.327049,-10.921602,9.795615,-5.031277,7.200982,-6.754969,2.663612,-4.380430,0.414055,blues


In [6]:
data.shape

(1000, 28)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[10]

array([-1.34212096, -1.40878536, -1.37442596, -1.68976734, -1.44634673,
       -0.80893346, -1.56168784,  1.96572022, -0.70660746, -0.49991917,
       -0.13900961, -1.03663238, -0.6158062 , -0.94724196,  0.02093777,
       -0.20708668, -0.30972376, -0.93535495,  0.11229713, -0.76437544,
        0.46671692, -0.1095389 ,  1.19204008, -0.18835474,  0.27617984,
       -1.56574893])

# Classification with Keras

## Building our Network

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 0s 373us/step - loss: 2.1270 - acc: 0.2687
Epoch 2/20
800/800 [==============================] - 0s 19us/step - loss: 1.8037 - acc: 0.3500
Epoch 3/20
800/800 [==============================] - 0s 17us/step - loss: 1.6149 - acc: 0.4037
Epoch 4/20
800/800 [==============================] - 0s 17us/step - loss: 1.4459 - acc: 0.4887
Epoch 5/20
800/800 [==============================] - 0s 17us/step - loss: 1.3095 - acc: 0.5363
Epoch 6/20
800/800 [==============================] - 0s 16us/step - loss: 1.1961 - acc: 0.5775
Epoch 7/20
800/800 [==============================] - 0s 17us/step - loss: 1.1084 - acc: 0.6238
Epoch 8/20
800/800 [==============================] - 0s 16us/step - loss: 1.0421 - acc: 0.6537
Epoch 9/20
800/800 [==============================] - 0s 18us/step - loss: 0.9705 - acc: 0.6687
Epoch 10/20
800/800 [==============================] - 0s 17us/step - loss: 0.9088 - ac

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 293us/step


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.67


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [19]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [20]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 583us/step - loss: 2.3202 - acc: 0.1050 - val_loss: 2.1771 - val_acc: 0.2750
Epoch 2/30
600/600 [==============================] - 0s 22us/step - loss: 2.1286 - acc: 0.3367 - val_loss: 2.0575 - val_acc: 0.3450
Epoch 3/30
600/600 [==============================] - 0s 21us/step - loss: 1.9866 - acc: 0.4067 - val_loss: 1.9308 - val_acc: 0.4150
Epoch 4/30
600/600 [==============================] - 0s 19us/step - loss: 1.8285 - acc: 0.4567 - val_loss: 1.8155 - val_acc: 0.4100
Epoch 5/30
600/600 [==============================] - 0s 20us/step - loss: 1.6930 - acc: 0.4417 - val_loss: 1.7060 - val_acc: 0.4350
Epoch 6/30
600/600 [==============================] - 0s 20us/step - loss: 1.5637 - acc: 0.4600 - val_loss: 1.6020 - val_acc: 0.4550
Epoch 7/30
600/600 [==============================] - 0s 19us/step - loss: 1.4436 - acc: 0.5150 - val_loss: 1.5398 - val_acc: 0.4600
Epoch 8/30
600/600 [==

In [21]:
results

[0.9398117017745972, 0.67]

## Predictions on Test Data

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

1.0

In [25]:
np.argmax(predictions[0])

6

## Pickling Model

In [26]:
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5")

In [ ]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

## Start Style Transfer

In [1]:

## Load model layers
# Model reconstruction from JSON file
with open('model_num.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_num.h5')

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [29]:
## Gram Matrix

In [30]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [31]:
## Style Loss

In [32]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [33]:
## Content Loss

In [34]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [35]:
## Total Variation Loss

In [36]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [39]:
from __future__ import print_function
from keras.preprocessing.image import load_img, save_img, img_to_array
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

In [40]:
## loss functions into scalar

In [ ]:
# combine these loss functions into a single scalar
loss = K.variable(0.0)
# import ipdb; ipdb.set_trace()
layer_features = outputs_dict['dense_5']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['dense_6',
                  'dense_7', 'dense_8',
                  'dense_9']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)